In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import re

In [3]:
df= pd.read_csv('/content/drive/MyDrive/유나다 코드 정리/빅콘테스트/1. Data/2023빅콘테스트_어드밴스드리그_예술의전당.csv')
df = df.loc[df['place']=='콘서트홀']
df.columns = ['나이','성별','멤버십종류1', '멤버십종류2', '멤버십종류3', '멤버십종류4', '멤버십종류5', '멤버십종류6', '예매 거래일자', '예매 거래시간',
'공연날짜', '공연시작시간', '좌석번호', '최종예매가격', '예매취소여부', '할인내역', '공연명코드', '선예매시작일','예매시작일','장르','공연장소','러닝타임','휴게시간','회원여부']

In [4]:
ticket = {0:'취소안함',2:'취소함'}
df['예매취소여부']=df['예매취소여부'].replace(ticket)

## 층_블록별 평균 가격 순위 파생변수

seat 변수를 사용하여 층_블록 변수 만들기

In [5]:
df['층'] = df['좌석번호'].str.split(' ').str.get(0)
df['블록'] = df['좌석번호'].str.split(' ').str.get(1)
df['자리'] = df['좌석번호'].str.split(' ').str.get(2)

def blocksplit(value):
  if value[:3] == 'BOX':
    return value
  else:
    return value[:3]

def blocksplitrow(value):
  if value[:3] == 'BOX':
    return 0
  else:
    return  value[3:]


df['열'] = df['블록'].map(blocksplitrow) # BOX는 열 없으니까 0, A블록21열 -> 21열
df['블록'] = df['블록'].map(blocksplit) # BOX9 -> 그래도 BOX9만, A블록21열 -> A블록만
df['층_블록'] = df['층'] + ' '+ df['블록']

In [6]:
from datetime import datetime
df['선예매시작일'] = df['선예매시작일'].fillna(0)
df['선예매시작일'] = df['선예매시작일'].astype(int)

df['예매시작일'] = df['예매시작일'].fillna(0)
df['예매시작일'] = df['예매시작일'].astype(int)

df['preorderandcancel'] = 0
df['preorderandcancel'] = 'F'  # 기본값 설정
# 각 조건에 따라 값을 설정합니다.
df.loc[(df['선예매시작일'] != 0) & (df['예매 거래일자'] < df['예매시작일']) & (df['예매취소여부'] == '취소함'), 'preorderandcancel'] = 'A'
df.loc[(df['선예매시작일'] != 0) & (df['예매 거래일자'] < df['예매시작일']) & (df['예매취소여부'] == '취소안함'), 'preorderandcancel'] = 'B'
df.loc[(df['선예매시작일'] != 0) & (df['예매 거래일자'] >= df['예매시작일']) & (df['예매취소여부'] == '취소함'), 'preorderandcancel'] = 'C'
df.loc[(df['선예매시작일'] != 0) & (df['예매 거래일자'] >= df['예매시작일']) & (df['예매취소여부'] == '취소안함'), 'preorderandcancel'] = 'D'
df.loc[(df['선예매시작일'] != 0) & (df['예매취소여부'] == '취소함'), 'preorderandcancel'] = 'E'


In [7]:
concertnum = list(df['공연명코드'].unique()) # 공연명 코드 값들 저장 (안 겹치게)
genre_list = list(df['장르'].unique()) # 장르 값들 저장 (안 겹치게)

concert_genre = [] # 공연명 코드에 해당하는 장르들 저장
for con in concertnum:
  concert_genre.append(df.loc[df['공연명코드']==con,'장르'].values[0])

In [8]:
final_concert = [] # 공연명코드가 담겨있는 리스트
final_genre = []  # 장르가 담겨있는 리스
for a in concertnum:
  if len(df.loc[df['공연명코드']==a,'장르'].unique()) ==1:
    final_concert.append(a)
    final_genre.append(df.loc[df['공연명코드']==a,'장르'].unique()[0])

가족극 = []
교향곡 = []
기타 = []
독주 = []
복합장르 = []
성악 = []
실내악 = []
오페라 = []
재즈 = []
콘서트 = []
크로스오버 = []
클래식 = []
합창 = []

i=0
for file in final_genre:
  if file =='가족극': # 전체 장르가 포함된 리스트에서 클래식의 값을 가진 값의 인덱스만 가져옴 ex( [교향곡, 클래식, 클래식 ]) -> [1,2]
    가족극.append(i)
    i+=1
  elif file =='교향곡':
    교향곡.append(i)
    i+=1
  elif file =='기타':
    기타.append(i)
    i+=1
  elif file =='독주':
    독주.append(i)
    i+=1
  elif file =='복합장르':
    복합장르.append(i)
    i+=1
  elif file =='성악':
    성악.append(i)
    i+=1
  elif file =='실내악': # 전체 장르가 포함된 리스트에서 클래식의 값을 가진 값의 인덱스만 가져옴 ex( [교향곡, 클래식, 클래식 ]) -> [1,2]
    실내악.append(i)
    i+=1
  elif file =='오페라':
    오페라.append(i)
    i+=1
  elif file =='재즈':
    재즈.append(i)
    i+=1
  elif file =='콘서트':
    콘서트.append(i)
    i+=1
  elif file =='크로스오버':
    크로스오버.append(i)
    i+=1
  elif file =='합창':
    합창.append(i)
    i+=1
  elif file =='클래식':
    클래식.append(i)
    i+=1
  else:
    i+=1

In [9]:
가족극concert = []
for a in 가족극:
  가족극concert.append(final_concert[a]) # 저장된 클래식이 있는 인덱스를 가져와서 공연번호 가져와서 저장하기

교향곡concert = []
for b in 교향곡:
  교향곡concert.append(final_concert[b])

기타concert = []
for b in 기타:
  기타concert.append(final_concert[b])


독주concert = []
for c in 독주:
  독주concert.append(final_concert[c])

복합장르concert = []
for f in 복합장르:
  복합장르concert.append(final_concert[f])

성악concert = []
for c in 성악:
  성악concert.append(final_concert[c])

실내악concert = []
for c in 실내악:
  실내악concert.append(final_concert[c])

오페라concert = []
for b in 오페라:
  오페라concert.append(final_concert[b])

재즈concert = []
for a in 재즈:
  재즈concert.append(final_concert[a])

콘서트concert = []
for d in 콘서트:
  콘서트concert.append(final_concert[d])

크로스오버concert = []
for e in 크로스오버:
  크로스오버concert.append(final_concert[e])


클래식concert = []
for d in 클래식:
  클래식concert.append(final_concert[d])

합창concert = []
for e in 합창:
  합창concert.append(final_concert[e])


In [10]:
orilist = ['1층 A블록',
 '1층 B블록',
 '1층 C블록',
 '1층 D블록',
 '1층 E블록',
 '2층 A블록',
 '2층 BOX1',
 '2층 BOX2',
 '2층 BOX3',
 '2층 BOX4',
 '2층 BOX5',
 '2층 BOX6',
 '2층 B블록',
 '2층 C블록',
 '2층 D블록',
 '2층 E블록',
 '3층 A블록',
 '3층 BOX10',
 '3층 BOX11',
 '3층 BOX12',
 '3층 BOX7',
 '3층 BOX8',
 '3층 BOX9',
 '3층 B블록',
 '3층 C블록',
 '3층 D블록',
 '3층 E블록',
 '3층 F블록',
 '3층 G블록',
 '3층 M블록',
 '3층 N블록',
 '합창석 F블록',
 '합창석 G블록',
 '합창석 H블록']

In [11]:
#####################  가족극 #####################
가족극df = pd.DataFrame({'층_블록':orilist})
for num in 가족극concert:
  if len(df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]) !=0:
    newdf = df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]
    seatlist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().index)
    valuelist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().values)
    ratiolist = list(np.round(valuelist / np.min(valuelist),3))
    testdf= pd.DataFrame({str(num):ratiolist,'층_블록':seatlist})
    가족극df = pd.merge(가족극df,testdf, how='outer',on='층_블록')

가족극df['총합'] = 가족극df.mean(axis=1)
가족극df = 가족극df.sort_values(by=['총합'], axis=0)
rangelist = range(0,34)
가족극df['순위'] = rangelist

small_가족극 = 가족극df.set_index(keys='층_블록')[['순위','총합']]
# 층_블록을 순위로
가족극_dict=small_가족극.loc['합창석 H블록':'2층 C블록', '순위'].to_dict()

#순위를 비율로
small_가족극2 = 가족극df.set_index(keys='순위')[['층_블록','총합']]
small_가족극d=small_가족극2.loc[0:6,'총합'].to_dict()

##################### 교향곡 #####################
교향곡df = pd.DataFrame({'층_블록':orilist})
for num in 교향곡concert:
  if len(df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]) !=0:
    newdf = df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]
    seatlist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().index)
    valuelist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().values)
    ratiolist = list(np.round(valuelist / np.min(valuelist),3))
    testdf= pd.DataFrame({str(num):ratiolist,'층_블록':seatlist})
    교향곡df = pd.merge(교향곡df,testdf, how='outer',on='층_블록')

교향곡df['총합'] = 교향곡df.mean(axis=1)
교향곡df = 교향곡df.sort_values(by=['총합'], axis=0)
rangelist = range(0,34)
교향곡df['순위'] = rangelist
small_교향곡 = 교향곡df.set_index(keys='층_블록')[['순위','총합']]

# 층_블록을 순위로
교향곡_dict=small_교향곡.loc['3층 M블록':'1층 B블록', '순위'].to_dict()
#순위를 비율로
small_교향곡2 = 교향곡df.set_index(keys='순위')[['층_블록','총합']]
교향곡_ratio_dict=small_교향곡2.loc[0:33,'총합'].to_dict()

##################### 기타 #####################

기타df = pd.DataFrame({'층_블록':orilist})
for num in 기타concert:
  if len(df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]) !=0:
    newdf = df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]
    seatlist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().index)
    valuelist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().values)
    ratiolist = list(np.round(valuelist / np.min(valuelist),3))
    testdf= pd.DataFrame({str(num):ratiolist,'층_블록':seatlist})
    기타df = pd.merge(기타df,testdf, how='outer',on='층_블록')

기타df['총합'] = 기타df.mean(axis=1)
기타df = 기타df.sort_values(by=['총합'], axis=0)
rangelist = range(0,34)
기타df['순위'] = rangelist
small_기타 = 기타df.set_index(keys='층_블록')[['총합','순위']]
기타_dict=small_기타.loc['합창석 G블록':'1층 C블록', '순위'].to_dict()
small_기타2 = 기타df.set_index(keys='순위')[['총합','층_블록']]
기타_ratio_dict=small_기타2.loc[0:5, '총합'].to_dict()


##################### 독주 #####################
독주df = pd.DataFrame({'층_블록':orilist})
for num in 독주concert:
  if len(df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]) !=0:
    newdf = df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]
    seatlist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().index)
    valuelist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().values)
    ratiolist = list(np.round(valuelist / np.min(valuelist),3))
    testdf= pd.DataFrame({str(num):ratiolist,'층_블록':seatlist})
    독주df = pd.merge(독주df,testdf, how='outer',on='층_블록')

독주df['총합'] = 독주df.mean(axis=1)
독주df = 독주df.sort_values(by=['총합'], axis=0)
rangelist = range(0,34)
독주df['순위'] = rangelist
small_독주 = 독주df.set_index(keys='층_블록')[['순위','총합']]
독주_dict=small_독주.loc['3층 BOX12':'1층 C블록', '순위'].to_dict()
small_독주2 = 독주df.set_index(keys='순위')[['층_블록','총합']]
독주_ratiodict=small_독주2.loc[0:33, '총합'].to_dict()

##################### 복합장르 #####################
복합장르df = pd.DataFrame({'층_블록':orilist})
for num in 복합장르concert:
  if len(df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]) !=0:
    newdf = df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]
    seatlist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().index)
    valuelist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().values)
    ratiolist = list(np.round(valuelist / np.min(valuelist),3))
    testdf= pd.DataFrame({str(num):ratiolist,'층_블록':seatlist})
    복합장르df = pd.merge(복합장르df,testdf, how='outer',on='층_블록')

복합장르df['총합'] = 복합장르df.mean(axis=1)
복합장르df = 복합장르df.sort_values(by=['총합'], axis=0)
rangelist = range(0,34)
복합장르df['순위'] = rangelist
small_복합장르 = 복합장르df.set_index(keys='층_블록')[['순위','총합']]
복합장르_dict=small_복합장르.loc['합창석 H블록':'1층 D블록', '순위'].to_dict()
small_복합장르2 = 복합장르df.set_index(keys='순위')[['층_블록','총합']]
복합장르_ratio_dict=small_복합장르2.loc[0:33, '총합'].to_dict()

##################### 성악  #####################
성악df = pd.DataFrame({'층_블록':orilist})
for num in 성악concert:
  if len(df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]) !=0:
    newdf = df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]
    seatlist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().index)
    valuelist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().values)
    ratiolist = list(np.round(valuelist / np.min(valuelist),3))
    testdf= pd.DataFrame({str(num):ratiolist,'층_블록':seatlist})
    성악df = pd.merge(성악df,testdf, how='outer',on='층_블록')

성악df['총합'] = 성악df.mean(axis=1)
성악df = 성악df.sort_values(by=['총합'], axis=0)
rangelist = range(0,34)
성악df['순위'] = rangelist
small_성악 = 성악df.set_index(keys='층_블록')[['순위','총합']]
성악_dict=small_성악.loc['합창석 G블록':'1층 C블록', '순위'].to_dict()
small_성악2 = 성악df.set_index(keys='순위')[['층_블록','총합']]
성악_ratio_dict=small_성악2.loc[0:33, '총합'].to_dict()

##################### 실악  #####################
실내악df = pd.DataFrame({'층_블록':orilist})
for num in 실내악concert:
  if len(df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]) !=0:
    newdf = df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]
    seatlist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().index)
    valuelist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().values)
    ratiolist = list(np.round(valuelist / np.min(valuelist),3))
    testdf= pd.DataFrame({str(num):ratiolist,'층_블록':seatlist})
    실내악df = pd.merge(실내악df,testdf, how='outer',on='층_블록')

실내악df['총합'] = 실내악df.mean(axis=1)
실내악df = 실내악df.sort_values(by=['총합'], axis=0)
rangelist = range(0,34)
실내악df['순위'] = rangelist
small_실내악 = 실내악df.set_index(keys='층_블록')[['순위','총합']]
실내악_dict=small_실내악.loc['3층 BOX12':'1층 C블록', '순위'].to_dict()
small_실내악2 = 실내악df.set_index(keys='순위')[['층_블록','총합']]
실내악_ratio_dict=small_실내악2.loc[0:33, '총합'].to_dict()

##################### 오페라  #####################
오페라df = pd.DataFrame({'층_블록':orilist})
for num in 오페라concert:
  if len(df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]) !=0:
    newdf = df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]
    seatlist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().index)
    valuelist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().values)
    ratiolist = list(np.round(valuelist / np.min(valuelist),3))
    testdf= pd.DataFrame({str(num):ratiolist,'층_블록':seatlist})
    오페라df = pd.merge(오페라df,testdf, how='outer',on='층_블록')

오페라df['총합'] = 오페라df.mean(axis=1)
오페라df = 오페라df.sort_values(by=['총합'], axis=0)
rangelist = range(0,34)
오페라df['순위'] = rangelist
small_오페라 = 오페라df.set_index(keys='층_블록')[['순위','총합']]
오페라_dict=small_오페라.loc['합창석 H블록':'합창석 F블록', '순위'].to_dict()
small_오페라2 = 오페라df.set_index(keys='순위')[['층_블록','총합']]
small_오페라=small_오페라2.loc[0:33, '총합'].to_dict()

##################### 재즈  #####################
재즈df = pd.DataFrame({'층_블록':orilist})
for num in 재즈concert:
  if len(df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]) !=0:
    newdf = df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]
    seatlist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().index)
    valuelist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().values)
    ratiolist = list(np.round(valuelist / np.min(valuelist),3))
    testdf= pd.DataFrame({str(num):ratiolist,'층_블록':seatlist})
    재즈df = pd.merge(재즈df,testdf, how='outer',on='층_블록')

재즈df['총합'] = 재즈df.mean(axis=1)
재즈df = 재즈df.sort_values(by=['총합'], axis=0)
rangelist = range(0,34)
재즈df['순위'] = rangelist
small_재즈 = 재즈df.set_index(keys='층_블록')[['순위','총합']]
재즈_dict=small_재즈.loc['3층 B블록':'2층 B블록', '순위'].to_dict()
small_재즈2 = 재즈df.set_index(keys='순위')[['층_블록','총합']]
재즈_dict=small_재즈2.loc[0:23, '총합'].to_dict()


##################### 콘서트  #####################
콘서트df = pd.DataFrame({'층_블록':orilist})
for num in 콘서트concert:
  if len(df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]) !=0:
    newdf = df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]
    seatlist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().index)
    valuelist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().values)
    ratiolist = list(np.round(valuelist / np.min(valuelist),3))
    testdf= pd.DataFrame({str(num):ratiolist,'층_블록':seatlist})
    콘서트df = pd.merge(콘서트df,testdf, how='outer',on='층_블록')

콘서트df['총합'] = 콘서트df.mean(axis=1)
콘서트df = 콘서트df.sort_values(by=['총합'], axis=0)
rangelist = range(0,34)
콘서트df['순위'] = rangelist
small_콘서트 = 콘서트df.set_index(keys='층_블록')[['순위','총합']]
콘서트_dict=small_콘서트.loc['3층 B블록':'1층 D블록', '순위'].to_dict()
small_콘서트2 = 콘서트df.set_index(keys='순위')[['층_블록','총합']]
콘서트_ratio_dict=small_콘서트2.loc[0:33, '총합'].to_dict()

##################### 크로스오버  #####################
크로스오버df = pd.DataFrame({'층_블록':orilist})
for num in 크로스오버concert:
  if len(df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]) !=0:
    newdf = df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]
    seatlist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().index)
    valuelist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().values)
    ratiolist = list(np.round(valuelist / np.min(valuelist),3))
    testdf= pd.DataFrame({str(num):ratiolist,'층_블록':seatlist})
    크로스오버df = pd.merge(크로스오버df,testdf, how='outer',on='층_블록')

크로스오버df['총합'] = 크로스오버df.mean(axis=1)
크로스오버df = 크로스오버df.sort_values(by=['총합'], axis=0)
rangelist = range(0,34)
크로스오버df['순위'] = rangelist
small_크로스오버 = 크로스오버df.set_index(keys='층_블록')[['순위','총합']]
크로스오버_dict=small_크로스오버.loc['3층 N블록':'1층 C블록', '순위'].to_dict()
small_크로스오버2 = 크로스오버df.set_index(keys='순위')[['층_블록','총합']]
크로스오버_ratio_dict=small_크로스오버2.loc[0:25, '총합'].to_dict()

##################### 클래식  #####################
클래식df = pd.DataFrame({'층_블록':orilist})
for num in 클래식concert:
  if len(df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]) !=0:
    newdf = df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]
    seatlist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().index)
    valuelist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().values)
    ratiolist = list(np.round(valuelist / np.min(valuelist),3))
    testdf= pd.DataFrame({str(num):ratiolist,'층_블록':seatlist})
    클래식df = pd.merge(클래식df,testdf, how='outer',on='층_블록')

클래식df['총합'] = 클래식df.mean(axis=1)
클래식df = 클래식df.sort_values(by=['총합'], axis=0)
rangelist = range(0,34)
클래식df['순위'] = rangelist
small_클래식 = 클래식df.set_index(keys='층_블록')[['순위','총합']]
클래식_dict=small_클래식.loc['합창석 F블록':'1층 C블록', '순위'].to_dict()
small_클래식2 = 클래식df.set_index(keys='순위')[['층_블록','총합']]
클래식_ratio_dict=small_클래식2.loc[0:33, '총합'].to_dict()

##################### 합창   #####################
합창df = pd.DataFrame({'층_블록':orilist})
for num in 합창concert:
  if len(df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]) !=0:
    newdf = df.loc[df['공연명코드']==num,].loc[df['최종예매가격']!=0,]
    seatlist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().index)
    valuelist = list(newdf.groupby(['층_블록'])['최종예매가격'].mean().values)
    ratiolist = list(np.round(valuelist / np.min(valuelist),3))
    testdf= pd.DataFrame({str(num):ratiolist,'층_블록':seatlist})
    합창df = pd.merge(합창df,testdf, how='outer',on='층_블록')
합창df['총합'] = 합창df.mean(axis=1)
합창df = 합창df.sort_values(by=['총합'], axis=0)
rangelist = range(0,34)
합창df['순위'] = rangelist
small_합창 = 합창df.set_index(keys='층_블록')[['순위','총합']]
합창_dict=small_합창.loc['합창석 G블록':'2층 BOX3', '순위'].to_dict()
small_합창2 = 합창df.set_index(keys='순위')[['층_블록','총합']]
합창_ratio_dict=small_합창2.loc[0:32, '총합'].to_dict()

df['좌석가격'] = df.loc[df['장르']=='합창','층_블록'].replace(합창_dict)
df['좌석가격'] = df.loc[df['장르']=='가족극','층_블록'].replace(가족극_dict)
df['좌석가격'] = df.loc[df['장르']=='교향곡','층_블록'].replace(교향곡_dict)
df['좌석가격'] = df.loc[df['장르']=='기타','층_블록'].replace(기타_dict)
df['좌석가격'] = df.loc[df['장르']=='독주','층_블록'].replace(독주_dict)
df['좌석가격'] = df.loc[df['장르']=='복합장르','층_블록'].replace(복합장르_dict)
df['좌석가격'] = df.loc[df['장르']=='성악','층_블록'].replace(성악_dict)
df['좌석가격'] = df.loc[df['장르']=='실내악','층_블록'].replace(실내악_dict)
df['좌석가격'] = df.loc[df['장르']=='오페라','층_블록'].replace(오페라_dict)
df['좌석가격'] = df.loc[df['장르']=='재즈','층_블록'].replace(재즈_dict)
df['좌석가격'] = df.loc[df['장르']=='콘서트','층_블록'].replace(콘서트_dict)
df['좌석가격'] = df.loc[df['장르']=='크로스오버','층_블록'].replace(크로스오버_dict)
df['좌석가격'] = df.loc[df['장르']=='클래식','층_블록'].replace(클래식_dict)

<ipython-input-11-adb271472ec3>:12: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  가족극df['총합'] = 가족극df.mean(axis=1)
<ipython-input-11-adb271472ec3>:36: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  교향곡df['총합'] = 교향곡df.mean(axis=1)
<ipython-input-11-adb271472ec3>:60: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  기타df['총합'] = 기타df.mean(axis=1)
<ipython-input-11-adb271472ec3>:81: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; i

## 장르별 등급 파생변수 생성

* 가장 많이 예매 된 좌석 순위별로 각 장르마다 좌석 순위 저장

In [12]:
seat_counts = df.groupby(['장르', '좌석번호']).size().unstack(fill_value=0)

def calculate_seat_rank(row):
    genre = row['장르']
    seat = row['좌석번호']
    if genre in seat_counts.index:
        seat_ranking = seat_counts.loc[genre].rank(ascending=False, method='min')
        return int(seat_ranking[seat])
    else:
        return np.nan

df['좌석순위'] = df.apply(calculate_seat_rank, axis=1)

print(df)

           나이   성별 멤버십종류1 멤버십종류2 멤버십종류3 멤버십종류4 멤버십종류5  멤버십종류6   예매 거래일자  \
0        50.0    F     골드     무료     그린    NaN    NaN     NaN  20220114   
4        50.0    M     무료    NaN    NaN    NaN    NaN     NaN  20220206   
5        30.0    F     블루     무료    NaN    NaN    NaN     NaN  20181124   
6         NaN  NaN    NaN    NaN    NaN    NaN    NaN     NaN  20191020   
7         NaN  NaN    NaN    NaN    NaN    NaN    NaN     NaN  20190613   
...       ...  ...    ...    ...    ...    ...    ...     ...       ...   
1920853  60.0    F     무료     그린    NaN    NaN    NaN     NaN  20210618   
1920855  50.0    M     무료    NaN    NaN    NaN    NaN     NaN  20230522   
1920856   NaN  NaN    NaN    NaN    NaN    NaN    NaN     NaN  20201009   
1920858   NaN  NaN    NaN    NaN    NaN    NaN    NaN     NaN  20200726   
1920864  20.0    F     무료   싹틔우미    NaN    NaN    NaN     NaN  20221224   

         예매 거래시간  ...  휴게시간  회원여부   층    블록  자리    열     층_블록  \
0           1512  ...    15     Y 

In [13]:
rank2 = df['좌석순위'].groupby(df['장르'])
rank2.unique()

장르
가족극                                       [1.0, 22.0, 2.0]
교향곡      [2107.0, 2213.0, 414.0, 596.0, 1633.0, 1952.0,...
기타                        [61.0, 1068.0, 2.0, 2352.0, 1.0]
독주       [1107.0, 601.0, 782.0, 33.0, 1827.0, 1366.0, 2...
복합장르     [987.0, 22.0, 644.0, 163.0, 387.0, 1444.0, 68....
성악       [414.0, 1400.0, 1655.0, 249.0, 1752.0, 98.0, 5...
실내악      [579.0, 1178.0, 204.0, 713.0, 343.0, 880.0, 13...
오페라      [1440.0, 1915.0, 10.0, 221.0, 822.0, 366.0, 56...
재즈                                  [18.0, 95.0, 1.0, 5.0]
콘서트      [1355.0, 804.0, 367.0, 1084.0, 1605.0, 561.0, ...
크로스오버                               [100.0, 9.0, 1.0, 3.0]
클래식      [640.0, 5.0, 397.0, 1928.0, 1252.0, 1075.0, 47...
합창       [14.0, 17.0, 1132.0, 1612.0, 504.0, 71.0, 1735...
Name: 좌석순위, dtype: object

In [14]:
rank2.nunique()

장르
가족극        3
교향곡      314
기타         5
독주        60
복합장르      15
성악        37
실내악       42
오페라       23
재즈         4
콘서트       24
크로스오버      4
클래식      158
합창        75
Name: 좌석순위, dtype: int64

* 장르별 좌석 순위를 기반으로 순위를 5개의 등급으로 나눈 후 장르별 등급 파생변수 생성하기 위해 각 장르의 좌석을 직접확인하여 5개의 등급으로 분리

In [15]:
rank2 = df.groupby('장르')['좌석순위'].apply(lambda x: sorted(list(set(x)), reverse=False)).reset_index()

print(rank2)

       장르                                               좌석순위
0     가족극                                   [1.0, 2.0, 22.0]
1     교향곡  [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...
2      기타                   [1.0, 2.0, 61.0, 1068.0, 2352.0]
3      독주  [1.0, 2.0, 3.0, 5.0, 6.0, 8.0, 10.0, 14.0, 15....
4    복합장르  [1.0, 2.0, 6.0, 15.0, 22.0, 46.0, 68.0, 107.0,...
5      성악  [1.0, 2.0, 3.0, 7.0, 14.0, 18.0, 30.0, 46.0, 7...
6     실내악  [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 8.0, 9.0, 10.0,...
7     오페라  [1.0, 2.0, 5.0, 6.0, 10.0, 16.0, 25.0, 43.0, 7...
8      재즈                             [1.0, 5.0, 18.0, 95.0]
9     콘서트  [1.0, 2.0, 4.0, 5.0, 10.0, 13.0, 21.0, 40.0, 7...
10  크로스오버                             [1.0, 3.0, 9.0, 100.0]
11    클래식  [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 8.0, 9.0, 10.0,...
12     합창  [1.0, 4.0, 5.0, 6.0, 8.0, 9.0, 14.0, 17.0, 25....


#### 1. 교향곡

In [16]:
genre1 = rank2[rank2['장르'] == '교향곡']['좌석순위'].iloc[0]
print(genre1)

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 13.0, 14.0, 16.0, 17.0, 19.0, 20.0, 22.0, 23.0, 25.0, 27.0, 28.0, 29.0, 30.0, 32.0, 33.0, 39.0, 40.0, 43.0, 44.0, 45.0, 47.0, 49.0, 50.0, 52.0, 54.0, 56.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 66.0, 68.0, 70.0, 72.0, 75.0, 77.0, 80.0, 83.0, 84.0, 89.0, 91.0, 92.0, 94.0, 95.0, 96.0, 99.0, 103.0, 106.0, 108.0, 113.0, 115.0, 122.0, 125.0, 129.0, 131.0, 134.0, 138.0, 140.0, 145.0, 149.0, 151.0, 155.0, 157.0, 162.0, 167.0, 173.0, 180.0, 188.0, 191.0, 197.0, 208.0, 214.0, 220.0, 222.0, 224.0, 228.0, 235.0, 243.0, 248.0, 255.0, 257.0, 264.0, 277.0, 280.0, 290.0, 294.0, 300.0, 310.0, 318.0, 329.0, 341.0, 350.0, 362.0, 372.0, 382.0, 396.0, 403.0, 414.0, 427.0, 438.0, 459.0, 474.0, 484.0, 505.0, 519.0, 537.0, 551.0, 572.0, 585.0, 596.0, 614.0, 629.0, 650.0, 672.0, 688.0, 711.0, 721.0, 735.0, 753.0, 785.0, 805.0, 822.0, 843.0, 859.0, 879.0, 897.0, 919.0, 939.0, 969.0, 993.0, 1018.0, 1036.0, 1061.0, 1077.0, 1110.0, 1132.0, 1150.0, 1171.0, 11

In [17]:
values = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 13.0, 14.0, 16.0, 17.0, 19.0, 20.0, 22.0, 23.0, 25.0, 27.0, 28.0, 29.0, 30.0, 32.0, 33.0, 39.0, 40.0, 43.0, 44.0, 45.0, 47.0, 49.0, 50.0, 52.0, 54.0, 56.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 66.0, 68.0, 70.0, 72.0, 75.0, 77.0, 80.0, 83.0, 84.0, 89.0, 91.0, 92.0, 94.0, 95.0, 96.0, 99.0, 103.0, 106.0, 108.0, 113.0, 115.0, 122.0, 125.0, 129.0, 131.0, 134.0, 138.0, 140.0, 145.0, 149.0, 151.0, 155.0, 157.0, 162.0, 167.0, 173.0, 180.0, 188.0, 191.0, 197.0, 208.0, 214.0, 220.0, 222.0, 224.0, 228.0, 235.0, 243.0, 248.0, 255.0, 257.0, 264.0, 277.0, 280.0, 290.0, 294.0, 300.0, 310.0, 318.0, 329.0, 341.0, 350.0, 362.0, 372.0, 382.0, 396.0, 403.0, 414.0, 427.0, 438.0, 459.0, 474.0, 484.0, 505.0, 519.0, 537.0, 551.0, 572.0, 585.0, 596.0, 614.0, 629.0, 650.0, 672.0, 688.0, 711.0, 721.0, 735.0, 753.0, 785.0, 805.0, 822.0, 843.0, 859.0, 879.0, 897.0, 919.0, 939.0, 969.0, 993.0, 1018.0, 1036.0, 1061.0, 1077.0, 1110.0, 1132.0, 1150.0, 1171.0, 1194.0, 1210.0, 1231.0, 1251.0, 1275.0, 1302.0, 1333.0, 1350.0, 1370.0, 1393.0, 1415.0, 1439.0, 1468.0, 1491.0, 1517.0, 1539.0, 1555.0, 1590.0, 1615.0, 1633.0, 1656.0, 1676.0, 1693.0, 1706.0, 1727.0, 1750.0, 1777.0, 1794.0, 1813.0, 1832.0, 1841.0, 1863.0, 1884.0, 1890.0, 1908.0, 1924.0, 1937.0, 1952.0, 1972.0, 1982.0, 1992.0, 2014.0, 2024.0, 2041.0, 2052.0, 2061.0, 2074.0, 2085.0, 2094.0, 2099.0, 2107.0, 2114.0, 2119.0, 2129.0, 2130.0, 2135.0, 2140.0, 2141.0, 2144.0, 2150.0, 2153.0, 2158.0, 2161.0, 2165.0, 2169.0, 2171.0, 2174.0, 2177.0, 2180.0, 2183.0, 2186.0, 2188.0, 2190.0, 2191.0, 2193.0, 2196.0, 2197.0, 2199.0, 2202.0, 2205.0, 2208.0, 2209.0, 2210.0, 2211.0, 2212.0, 2213.0, 2214.0, 2216.0, 2217.0, 2218.0, 2220.0, 2223.0, 2224.0, 2225.0, 2227.0, 2230.0, 2231.0, 2235.0, 2236.0, 2240.0, 2243.0, 2248.0, 2253.0, 2259.0, 2264.0, 2273.0, 2274.0, 2279.0, 2286.0, 2291.0, 2298.0, 2306.0, 2313.0, 2321.0, 2326.0, 2331.0, 2338.0, 2347.0, 2355.0, 2364.0, 2371.0, 2376.0, 2386.0, 2389.0, 2396.0, 2404.0, 2410.0, 2426.0, 2428.0, 2434.0, 2442.0, 2449.0, 2451.0, 2458.0, 2459.0, 2461.0, 2464.0, 2467.0, 2470.0, 2471.0, 2476.0, 2478.0, 2482.0, 2483.0, 2484.0, 2485.0, 2486.0, 2487.0, 2488.0, 2489.0, 2490.0, 2491.0, 2492.0, 2493.0, 2494.0, 2495.0, 2496.0, 2497.0, 2499.0, 2501.0, 2502.0, 2505.0, 2506.0]

value_63 = values[62]
value_126 = values[125]
value_189 = values[188]
value_252 = values[251]
value_314 = values[313]

print(value_63, value_126, value_189, value_252, value_314)

113.0 650.0 1952.0 2243.0 2506.0


#### 2. 독주

In [18]:
genre2 = rank2[rank2['장르'] == '독주']['좌석순위'].iloc[0]
print(genre2)

[1.0, 2.0, 3.0, 5.0, 6.0, 8.0, 10.0, 14.0, 15.0, 18.0, 22.0, 27.0, 33.0, 46.0, 60.0, 72.0, 85.0, 100.0, 120.0, 145.0, 175.0, 197.0, 227.0, 259.0, 295.0, 346.0, 413.0, 475.0, 542.0, 601.0, 663.0, 722.0, 782.0, 838.0, 904.0, 979.0, 1042.0, 1107.0, 1199.0, 1275.0, 1366.0, 1451.0, 1528.0, 1620.0, 1714.0, 1827.0, 1937.0, 2034.0, 2111.0, 2206.0, 2290.0, 2347.0, 2399.0, 2432.0, 2468.0, 2482.0, 2495.0, 2499.0, 2502.0, 2504.0]


In [19]:
value_1 = genre2[11]
value_2 = genre2[23]
value_3 = genre2[35]
value_4 = genre2[47]
value_5 = genre2[59]

print(value_1, value_2, value_3, value_4, value_5)

27.0 259.0 979.0 2034.0 2504.0


#### 3. 복합장르

In [20]:
genre3 = rank2[rank2['장르'] == '복합장르']['좌석순위'].iloc[0]
print(genre3)

[1.0, 2.0, 6.0, 15.0, 22.0, 46.0, 68.0, 107.0, 163.0, 240.0, 387.0, 644.0, 987.0, 1444.0, 2093.0]


In [21]:
value_1 = genre3[2]
value_2 = genre3[5]
value_3 = genre3[8]
value_4 = genre3[11]
value_5 = genre3[14]

print(value_1, value_2, value_3, value_4, value_5)

6.0 46.0 163.0 644.0 2093.0


#### 4.성악

In [22]:
genre4 = rank2[rank2['장르'] == '성악']['좌석순위'].iloc[0]
print(genre4)

[1.0, 2.0, 3.0, 7.0, 14.0, 18.0, 30.0, 46.0, 72.0, 98.0, 135.0, 195.0, 249.0, 323.0, 414.0, 525.0, 648.0, 764.0, 897.0, 1028.0, 1161.0, 1289.0, 1400.0, 1537.0, 1655.0, 1752.0, 1845.0, 1933.0, 2015.0, 2070.0, 2130.0, 2174.0, 2204.0, 2227.0, 2275.0, 2340.0, 2445.0]


In [23]:
value_1 = genre4[6]
value_2 = genre4[14]
value_3 = genre4[22]
value_4 = genre4[30]
value_5 = genre4[36]

print(value_1, value_2, value_3, value_4, value_5)

30.0 414.0 1400.0 2130.0 2445.0


#### 5. 실내악

In [24]:
genre5 = rank2[rank2['장르'] == '실내악']['좌석순위'].iloc[0]
print(genre5)

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 8.0, 9.0, 10.0, 11.0, 18.0, 20.0, 27.0, 32.0, 39.0, 51.0, 74.0, 103.0, 127.0, 156.0, 204.0, 262.0, 343.0, 441.0, 579.0, 713.0, 880.0, 1028.0, 1178.0, 1315.0, 1458.0, 1589.0, 1714.0, 1807.0, 1890.0, 1963.0, 2029.0, 2122.0, 2238.0, 2324.0, 2403.0, 2462.0]


In [25]:
value_1 = genre5[7]
value_2 = genre5[16]
value_3 = genre5[25]
value_4 = genre5[34]
value_5 = genre5[41]

print(value_1, value_2, value_3, value_4, value_5)

9.0 74.0 713.0 1890.0 2462.0


#### 6. 오페라

In [26]:
genre6 = rank2[rank2['장르'] == '오페라']['좌석순위'].iloc[0]
print(genre6)

[1.0, 2.0, 5.0, 6.0, 10.0, 16.0, 25.0, 43.0, 78.0, 117.0, 221.0, 366.0, 567.0, 822.0, 1111.0, 1440.0, 1697.0, 1915.0, 2104.0, 2185.0, 2208.0, 2210.0, 2220.0]


In [27]:
value_1 = genre6[3]
value_2 = genre6[8]
value_3 = genre6[13]
value_4 = genre6[18]
value_5 = genre6[22]

print(value_1, value_2, value_3, value_4, value_5)

6.0 78.0 822.0 2104.0 2220.0


#### 7. 콘서트

In [28]:
genre7 = rank2[rank2['장르'] == '콘서트']['좌석순위'].iloc[0]
print(genre7)

[1.0, 2.0, 4.0, 5.0, 10.0, 13.0, 21.0, 40.0, 77.0, 157.0, 254.0, 367.0, 561.0, 804.0, 1084.0, 1355.0, 1605.0, 1819.0, 1967.0, 2076.0, 2154.0, 2251.0, 2358.0, 2469.0]


In [29]:
value_1 = genre7[3]
value_2 = genre7[8]
value_3 = genre7[14]
value_4 = genre7[19]
value_5 = genre7[23]

print(value_1, value_2, value_3, value_4, value_5)

5.0 77.0 1084.0 2076.0 2469.0


#### 8. 클래식

In [30]:
genre8 = rank2[rank2['장르'] == '클래식']['좌석순위'].iloc[0]
print(genre8)

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 8.0, 9.0, 10.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 23.0, 26.0, 33.0, 38.0, 39.0, 42.0, 46.0, 53.0, 57.0, 60.0, 66.0, 73.0, 77.0, 81.0, 87.0, 90.0, 96.0, 102.0, 108.0, 113.0, 125.0, 132.0, 145.0, 157.0, 172.0, 190.0, 208.0, 216.0, 230.0, 254.0, 274.0, 295.0, 312.0, 335.0, 353.0, 375.0, 397.0, 420.0, 448.0, 473.0, 506.0, 543.0, 582.0, 610.0, 640.0, 679.0, 708.0, 751.0, 799.0, 841.0, 876.0, 921.0, 964.0, 999.0, 1036.0, 1075.0, 1118.0, 1165.0, 1200.0, 1252.0, 1306.0, 1359.0, 1397.0, 1439.0, 1486.0, 1535.0, 1583.0, 1631.0, 1678.0, 1709.0, 1752.0, 1790.0, 1829.0, 1864.0, 1903.0, 1928.0, 1952.0, 1979.0, 1997.0, 2018.0, 2039.0, 2052.0, 2069.0, 2079.0, 2095.0, 2110.0, 2123.0, 2140.0, 2156.0, 2164.0, 2178.0, 2190.0, 2200.0, 2225.0, 2236.0, 2249.0, 2258.0, 2271.0, 2281.0, 2293.0, 2309.0, 2322.0, 2334.0, 2344.0, 2361.0, 2368.0, 2377.0, 2390.0, 2400.0, 2411.0, 2421.0, 2427.0, 2435.0, 2439.0, 2444.0, 2446.0, 2454.0, 2461.0, 2465.0, 2466.0, 2473.0, 2475.0, 2477.0, 2480

In [31]:
value_1 = genre8[31]
value_2 = genre8[63]
value_3 = genre8[95]
value_4 = genre8[127]
value_5 = genre8[157]

print(value_1, value_2, value_3, value_4, value_5)

96.0 799.0 2039.0 2435.0 2508.0


#### 9. 합창

In [32]:
genre9 = rank2[rank2['장르'] == '합창']['좌석순위'].iloc[0]
print(genre9)

[1.0, 4.0, 5.0, 6.0, 8.0, 9.0, 14.0, 17.0, 25.0, 37.0, 50.0, 57.0, 71.0, 94.0, 110.0, 129.0, 165.0, 200.0, 255.0, 319.0, 374.0, 436.0, 504.0, 576.0, 655.0, 732.0, 845.0, 948.0, 1043.0, 1132.0, 1241.0, 1339.0, 1414.0, 1496.0, 1557.0, 1612.0, 1676.0, 1735.0, 1802.0, 1856.0, 1902.0, 1936.0, 1972.0, 2002.0, 2029.0, 2061.0, 2094.0, 2118.0, 2148.0, 2165.0, 2185.0, 2208.0, 2231.0, 2250.0, 2264.0, 2279.0, 2308.0, 2329.0, 2359.0, 2384.0, 2402.0, 2420.0, 2439.0, 2457.0, 2467.0, 2475.0, 2479.0, 2483.0, 2484.0, 2486.0, 2493.0, 2495.0, 2499.0, 2502.0, 2506.0]


In [33]:
value_1 = genre9[14]
value_2 = genre9[29]
value_3 = genre9[44]
value_4 = genre9[59]
value_5 = genre9[74]

print(value_1, value_2, value_3, value_4, value_5)

110.0 1132.0 2029.0 2384.0 2506.0


In [34]:
def calculate_seat_grade(row):
    if row['장르'] == '가족극':
        if row['좌석순위'] == 1.0:
            return 1
        elif row['좌석순위'] == 2.0:
            return 2
        elif row['좌석순위'] == 22.0:
            return 3
    elif row['장르'] == '재즈':
        if row['좌석순위'] == 1.0:
            return 1
        elif row['좌석순위'] == 5.0:
            return 2
        elif row['좌석순위'] == 18.0:
            return 3
        elif row['좌석순위'] == 95.0:
            return 4
    elif row['장르'] == '크로스오버':
        if row['좌석순위'] == 1.0:
            return 1
        elif row['좌석순위'] == 3.0:
            return 2
        elif row['좌석순위'] == 9.0:
            return 3
        elif row['좌석순위'] == 100.0:
            return 4
    elif row['장르'] == '기타':
        if row['좌석순위'] == 1.0:
            return 1
        elif row['좌석순위'] == 2.0:
            return 2
        elif row['좌석순위'] == 61.0:
            return 3
        elif row['좌석순위'] == 1068.0:
            return 4
        elif row['좌석순위'] == 2352.0:
            return 5
    elif row['장르'] == '교향곡':
        if 1.0 <= row['좌석순위'] <= 113.0:
            return 1
        elif 114.0 <= row['좌석순위'] <= 650.0:
            return 2
        elif 651.0 <= row['좌석순위'] <= 1952.0:
            return 3
        elif 1953.0 <= row['좌석순위'] <= 2243.0:
            return 4
        elif 2244.0 <= row['좌석순위'] <= 2506.0:
            return 5
    elif row['장르'] == '독주':
        if 1.0 <= row['좌석순위'] <= 27.0:
            return 1
        elif 28.0 <= row['좌석순위'] <= 259.0:
            return 2
        elif 260.0 <= row['좌석순위'] <= 979.0:
            return 3
        elif 980.0 <= row['좌석순위'] <= 2034.0:
            return 4
        elif 2035.0 <= row['좌석순위'] <= 2504.0:
            return 5
    elif row['장르'] == '복합장르':
        if 1 <= row['좌석순위'] <= 6.0:
            return 1
        elif 7.0 <= row['좌석순위'] <=46.0:
            return 2
        elif 47.0 <= row['좌석순위'] <= 163.0:
            return 3
        elif 164.0 <= row['좌석순위'] <= 644.0:
            return 4
        elif 645.0 <= row['좌석순위'] <= 2093.0:
            return 5
    elif row['장르'] == '성악':
        if 1 <= row['좌석순위'] <= 30.0:
            return 1
        elif 31.0 <= row['좌석순위'] <= 414.0:
            return 2
        elif 415.0 <= row['좌석순위'] <= 1400.0:
            return 3
        elif 1401.0 <= row['좌석순위'] <= 2130.0:
            return 4
        elif 2131.0 <= row['좌석순위'] <= 2445.0:
            return 5
    elif row['장르'] == '실내악':
        if 1 <= row['좌석순위'] <= 9.0:
            return 1
        elif 10.0 <= row['좌석순위'] <= 74.0:
            return 2
        elif 75.0 <= row['좌석순위'] <= 713.0:
            return 3
        elif 714.0 <= row['좌석순위'] <= 1890.0:
            return 4
        elif 1891.0 <= row['좌석순위'] <= 2462.0:
            return 5
    elif row['장르'] == '오페라':
        if 1 <= row['좌석순위'] <= 6.0:
            return 1
        elif 7.0 <= row['좌석순위'] <=78.0:
            return 2
        elif 79.0 <= row['좌석순위'] <= 822.0:
            return 3
        elif 823.0 <= row['좌석순위'] <= 2104.0:
            return 4
        elif 2105.0 <= row['좌석순위'] <= 2220.0:
            return 5
    elif row['장르'] == '콘서트':
        if 1 <= row['좌석순위'] <= 5.0:
            return 1
        elif 6.0 <= row['좌석순위'] <= 77.0:
            return 2
        elif 78.0 <= row['좌석순위'] <= 1084.0:
            return 3
        elif 1085.0 <= row['좌석순위'] <= 2076.0:
            return 4
        elif 2077.0 <= row['좌석순위'] <= 2469.0:
            return 5
    elif row['장르'] == '클래식':
        if 1 <= row['좌석순위'] <= 96.0:
            return 1
        elif 97.0 <= row['좌석순위'] <= 799.0:
            return 2
        elif 800.0 <= row['좌석순위'] <= 2039.0:
            return 3
        elif 2040.0 <= row['좌석순위'] <= 2435.0:
            return 4
        elif 2436.0 <= row['좌석순위'] <= 2508.0:
            return 5
    elif row['장르'] == '합창':
        if 1 <= row['좌석순위'] <= 110.0:
            return 1
        elif 111.0 <= row['좌석순위'] <= 1132.0:
            return 2
        elif 1133.0 <= row['좌석순위'] <= 2029.0:
            return 3
        elif 2030.0 <= row['좌석순위'] <= 2384.0:
            return 4
        elif 2385.0 <= row['좌석순위'] <= 2506.0:
            return 5



df['좌석등급'] = df.apply(calculate_seat_grade, axis=1)
df.head()

,나이,성별,멤버십종류1,멤버십종류2,멤버십종류3,멤버십종류4,멤버십종류5,멤버십종류6,예매 거래일자,예매 거래시간,...,회원여부,층,블록,자리,열,층_블록,preorderandcancel,좌석가격,좌석순위,좌석등급
0,50.0,F,골드,무료,그린,NaN,NaN,NaN,20220114,1512,...,Y,3층,BOX9,10,0,3층 BOX9,E,NaN,2107.0,4.0
4,50.0,M,무료,NaN,NaN,NaN,NaN,NaN,20220206,1615,...,Y,1층,B블록,7,12열,1층 B블록,D,NaN,1107.0,4.0
5,30.0,F,블루,무료,NaN,NaN,NaN,NaN,20181124,1145,...,Y,1층,A블록,1,2열,1층 A블록,E,NaN,2213.0,4.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20191020,1459,...,N,3층,E블록,8,4열,3층 E블록,F,NaN,414.0,2.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190613,954,...,N,2층,D블록,4,8열,2층 D블록,D,NaN,596.0,2.0


## 선예매 파생변수 추가

In [35]:
df['선예매시작일'] = df['선예매시작일'].fillna(0)
df['선예매시작일'] = df['선예매시작일'].astype(int)

df['예매시작일'] = df['예매시작일'].fillna(0)
df['예매시작일'] = df['예매시작일'].astype(int)

In [36]:
prev_yes_df = df.loc[df['선예매시작일']!=0].loc[df ['예매 거래일자'] < df['예매시작일'],]
prev_no_df = df.loc[df['선예매시작일']!=0].loc[df ['예매 거래일자'] >= df['예매시작일'],]
aft_df = df.loc[df['선예매시작일']==0,]

In [37]:
prev_yes_df.loc[prev_yes_df['예매취소여부']=='취소함','preorderandcancel'] = 'A'
prev_yes_df.loc[prev_yes_df['예매취소여부']=='취소안함','preorderandcancel'] = 'B'
prev_no_df.loc[prev_no_df['예매취소여부']=='취소함','preorderandcancel'] = 'C'
prev_no_df.loc[prev_no_df['예매취소여부']=='취소안함','preorderandcancel'] = 'D'
aft_df.loc[aft_df['예매취소여부']=='취소함','preorderandcancel'] = 'E'
aft_df.loc[aft_df['예매취소여부']=='취소안함','preorderandcancel'] = 'F'

In [38]:
df = pd.concat([prev_yes_df,prev_no_df,aft_df],axis=0)

In [39]:
df.head()

,나이,성별,멤버십종류1,멤버십종류2,멤버십종류3,멤버십종류4,멤버십종류5,멤버십종류6,예매 거래일자,예매 거래시간,...,회원여부,층,블록,자리,열,층_블록,preorderandcancel,좌석가격,좌석순위,좌석등급
0,50.0,F,골드,무료,그린,NaN,NaN,NaN,20220114,1512,...,Y,3층,BOX9,10,0,3층 BOX9,A,NaN,2107.0,4.0
18,70.0,M,골드,무료,NaN,NaN,NaN,NaN,20191220,1106,...,Y,2층,BOX2,2,0,2층 BOX2,B,21.0,5.0,1.0
27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190704,1120,...,N,3층,G블록,8,5열,3층 G블록,B,2.0,1928.0,3.0
58,30.0,F,무료,그린,NaN,NaN,NaN,NaN,20230203,1400,...,Y,1층,E블록,3,19열,1층 E블록,A,NaN,1539.0,3.0
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220905,956,...,N,1층,A블록,1,8열,1층 A블록,A,NaN,2141.0,4.0


## 3차원 좌표 파생변수 생성

* 3차원 좌표를 생성하기 위해 좌석을 구역, 열, 층마다 분리

In [40]:
df = df.reset_index()

In [41]:
# 좌석 분류
df['floor'] = 0
df['block'] = 0
df['row'] = 0
df['col'] = 0

for i in range(len(df)):
  target = re.findall(r'\d+', df['좌석번호'][i])
  if (len(target) == 2):
    df['floor'][i] = -1 #합창석은 -1층으로 표기
    df['block'][i] = df['좌석번호'][i][4]
    df['row'][i] = int(target[0])
    df['col'][i] = int(target[1])

  elif(df['좌석번호'].iloc[i][3:6] == 'BOX'):
    df['floor'][i] = int(target[0])
    df['block'][i] = 'BOX' #박스석의 경우 블록 대신 BOX로 표기
    df['row'][i] = int(target[1])
    df['col'][i] = int(target[2])
  else:
    df['floor'][i] = int(target[0])
    df['block'][i] = df['좌석번호'][i][3]
    df['row'][i] = int(target[1])
    df['col'][i] = int(target[2])

df.head()

<ipython-input-41-5d7c1468836e>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['floor'][i] = int(target[0])
<ipython-input-41-5d7c1468836e>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['block'][i] = 'BOX' #박스석의 경우 블록 대신 BOX로 표기
<ipython-input-41-5d7c1468836e>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['row'][i] = int(target[1])
<ipython-input-41-5d7c1468836e>:19: SettingWithCopyWarning: 
A value is trying to be 

,index,나이,성별,멤버십종류1,멤버십종류2,멤버십종류3,멤버십종류4,멤버십종류5,멤버십종류6,예매 거래일자,...,열,층_블록,preorderandcancel,좌석가격,좌석순위,좌석등급,floor,block,row,col
0,0,50.0,F,골드,무료,그린,NaN,NaN,NaN,20220114,...,0,3층 BOX9,A,NaN,2107.0,4.0,3,BOX,9,10
1,18,70.0,M,골드,무료,NaN,NaN,NaN,NaN,20191220,...,0,2층 BOX2,B,21.0,5.0,1.0,2,BOX,2,2
2,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190704,...,5열,3층 G블록,B,2.0,1928.0,3.0,3,G,5,8
3,58,30.0,F,무료,그린,NaN,NaN,NaN,NaN,20230203,...,19열,1층 E블록,A,NaN,1539.0,3.0,1,E,19,3
4,64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220905,...,8열,1층 A블록,A,NaN,2141.0,4.0,1,A,8,1


* 좌석 좌표 생성

In [42]:
box_z = [-1, 6.6, 6.2, 5.8, 5.8, 6.2, 6.6, 9.6, 9.2, 8.8, 8.8, 9.2, 9.6]
합창_y = [-1, 7.2, 7.8, 8.4, 0]
box_y = [-1, -11.98, -12.52, -13.06, -13.06, -12.52, -11.98, -16.92, -17.46, -18, -18, -17.46, -16.92]
합창_x = [-1, 14.5, 16, 17.5, 19]
int(df['row'][0])*3

27

In [43]:
df['x'] = 0
df['y'] = 0
df['z'] = 0

for i in range(len(df)):

  # x좌표 설정
  if ((df['floor'][i] == -1) & (df['block'][i] == 'G')):
    x = (df['col'][i] - 합창_x[df['row'][i]]) * 0.27
  elif ((df['floor'][i] == -1) & (df['block'][i] == 'H')):
    x = -8.28 - (df['row'][i]-1)*0.27
  elif ((df['floor'][i] == -1) & (df['block'][i] == 'F')):
    x = 7.2 + (df['row'][i]-1)*0.27

  elif ((df['floor'][i] == 1) & (df['block'][i] == 'C')):
    x = (df['col'][i] - 8) * 0.27
  elif ((df['floor'][i] == 1) & (df['block'][i] == 'B')):
    x = -6.21 + df['col'][i] * 0.27
  elif ((df['floor'][i] == 1) & (df['block'][i] == 'A')):
    x = -9.45 + df['col'][i] * 0.27
  elif ((df['floor'][i] == 1) & (df['block'][i] == 'D')):
    x = 2.43 + df['col'][i] * 0.27
  elif ((df['floor'][i] == 1) & (df['block'][i] == 'E')):
    x = 6.21 + df['col'][i] * 0.27

  elif ((df['floor'][i] == 2) & (df['block'][i] == 'C')):
    x = (df['col'][i] - 8) * 0.27
  elif ((df['floor'][i] == 2) & (df['block'][i] == 'B')):
    x = -5.4 + df['col'][i] * 0.27
  elif ((df['floor'][i] == 2) & (df['block'][i] == 'A')):
    x = -10.26 + df['col'][i] * 0.27
  elif ((df['floor'][i] == 2) & (df['block'][i] == 'D')):
    x = 1.89 + df['col'][i] * 0.27
  elif ((df['floor'][i] == 2) & (df['block'][i] == 'E')):
    x = 5.4 + df['col'][i] * 0.27

  elif ((df['floor'][i] == 3) & (df['block'][i] == 'D')):
    x = (df['col'][i] - 8) * 0.27
  elif ((df['floor'][i] == 3) & (df['block'][i] == 'C')):
    x = -5.4 + df['col'][i] * 0.27
  elif ((df['floor'][i] == 3) & (df['block'][i] == 'B')):
    x = -7.83 + df['col'][i] * 0.27
  elif ((df['floor'][i] == 3) & (df['block'][i] == 'A')):
    x = -11.61 + df['col'][i] * 0.27
  elif ((df['floor'][i] == 3) & (df['block'][i] == 'E')):
    x = 1.89 + df['col'][i] * 0.27
  elif ((df['floor'][i] == 3) & (df['block'][i] == 'F')):
    x = 5.4 + df['col'][i] * 0.27
  elif ((df['floor'][i] == 3) & (df['block'][i] == 'G')):
    x = 9.18 + df['col'][i] * 0.27

  elif ((df['floor'][i] == 3) & (df['block'][i] == 'N')):
    x = -5.4 + df['col'][i] * 0.27
  elif ((df['floor'][i] == 3) & (df['block'][i] == 'M')):
    x = 1.89 + df['col'][i] * 0.27

  elif ((df['block'][i] == 'BOX') & (df['floor'][i] == 2)):
    if (df['row'][i] == 1):
      x = -11.61
    elif (df['row'][i] == 2):
      x = -11.07
    elif (df['row'][i] == 3):
      x = -10.53

    if (df['row'][i] == 6):
      x = 12.15
    elif (df['row'][i] == 5):
      x = 11.61
    elif (df['row'][i] == 4):
      x = 11.07

  elif ((df['block'][i] == 'BOX') & (df['floor'][i] == 3)):
    if (df['row'][i] == 7):
      x = -11.61
    elif (df['row'][i] == 8):
      x = -12.15
    elif (df['row'][i] == 9):
      x = -12.69

    if (df['row'][i] == 10):
      x = 11.61
    elif (df['row'][i] == 11):
      x = 12.15
    elif (df['row'][i] == 12):
      x = 12.69

  # z 좌표 설정
  if (df['floor'][i] ==1):
    z = df['row'][i]*0.18
  elif (df['floor'][i] == -1):
    z = 5.5 + df['row'][i]*0.2

  elif ((df['block'][i] == 'BOX') & (df['floor'][i] == 2)):
    z = box_z[df['row'][i]]

  elif ((df['block'][i] == 'BOX') & (df['floor'][i] == 3)):
    z = box_z[df['row'][i]]

  elif (df['floor'][i] == 2):
    z = 5.5 + df['row'][i]*0.2

  elif ((df['floor'][i] == 3) & (df['block'][i] == 'N')):
    z = 11.5 + df['row'][i]*0.2
  elif ((df['floor'][i] == 3) & (df['block'][i] == 'M')):
    z = 11.5 + df['row'][i]*0.2

  elif (df['floor'][i] == 3):
    z = 9.5 + df['row'][i]*0.2


  # y 좌표 설정
  if ((df['floor'][i] == -1) & (df['block'][i] == 'G')):
    y = 7.2 + df['row'][i]*0.27
  elif (df['floor'][i] == -1):
    y = 합창_y[df['row'][i]] - (df['col'][i]-1)*0.27

  elif ((df['block'][i] == 'BOX') & (df['floor'][i] == 2)):
    y = box_y[df['row'][i]] + ((df['col'][i]+1)//2 - 1)*0.27

  elif ((df['block'][i] == 'BOX') & (df['floor'][i] == 3)):
    y = box_y[df['row'][i]] + ((df['col'][i]+2)//3 - 1)*0.27

  elif (df['floor'][i] == 1):
    y = -7.2 - (df['row'][i] - 1) * 0.27
  elif (df['floor'][i] == 2):
    y = -13.6 - (df['row'][i] - 1) * 0.27

  elif ((df['floor'][i] == 3) & (df['block'][i] == 'N')):
    y = -21.24 - (df['row'][i] - 1) * 0.27
  elif ((df['floor'][i] == 3) & (df['block'][i] == 'M')):
    y = -21.24 - (df['row'][i] - 1) * 0.27

  elif (df['floor'][i] == 3):
    y = -18.54 - (df['row'][i] - 1) * 0.27

  df['x'][i] = x
  df['y'][i] = y
  df['z'][i] = z

df

<ipython-input-43-5340d7d8e4ce>:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['x'][i] = x
<ipython-input-43-5340d7d8e4ce>:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['y'][i] = y
<ipython-input-43-5340d7d8e4ce>:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['z'][i] = z


,index,나이,성별,멤버십종류1,멤버십종류2,멤버십종류3,멤버십종류4,멤버십종류5,멤버십종류6,예매 거래일자,...,좌석가격,좌석순위,좌석등급,floor,block,row,col,x,y,z
0,0,50.0,F,골드,무료,그린,NaN,NaN,NaN,20220114,...,NaN,2107.0,4.0,3,BOX,9,10,-12.69,-17.19,8.80
1,18,70.0,M,골드,무료,NaN,NaN,NaN,NaN,20191220,...,21.0,5.0,1.0,2,BOX,2,2,-11.07,-12.52,6.20
2,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190704,...,2.0,1928.0,3.0,3,G,5,8,11.34,-19.62,10.50
3,58,30.0,F,무료,그린,NaN,NaN,NaN,NaN,20230203,...,NaN,1539.0,3.0,1,E,19,3,7.02,-12.06,3.42
4,64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220905,...,NaN,2141.0,4.0,1,A,8,1,-9.18,-9.09,1.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096963,1920843,60.0,F,골드,무료,NaN,NaN,NaN,NaN,20181223,...,NaN,1794.0,3.0,3,N,1,8,-3.24,-21.24,11.70
1096964,1920845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220602,...,NaN,200.0,2.0,1,B,21,8,-4.05,-12.60,3.78
1096965,1920846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20191005,...,NaN,61.0,3.0,3,N,1,8,-3.24,-21.24,11.70
1096966,1920856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20201009,...,32.0,876.0,3.0,1,D,16,12,5.67,-11.25,2.88


## 거리 파생변수 생성

In [44]:
df['dist'] = 0

for i in range(len(df)):
  distance = np.sqrt(df['x'][i]**2 + df['y'][i]**2 + df['z'][i]**2)
  df['dist'][i] = distance

df.head()

<ipython-input-44-fe85ff263652>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dist'][i] = distance


,index,나이,성별,멤버십종류1,멤버십종류2,멤버십종류3,멤버십종류4,멤버십종류5,멤버십종류6,예매 거래일자,...,좌석순위,좌석등급,floor,block,row,col,x,y,z,dist
0,0,50.0,F,골드,무료,그린,NaN,NaN,NaN,20220114,...,2107.0,4.0,3,BOX,9,10,-12.69,-17.19,8.80,23.107838
1,18,70.0,M,골드,무료,NaN,NaN,NaN,NaN,20191220,...,5.0,1.0,2,BOX,2,2,-11.07,-12.52,6.20,17.825131
2,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190704,...,1928.0,3.0,3,G,5,8,11.34,-19.62,10.50,24.975788
3,58,30.0,F,무료,그린,NaN,NaN,NaN,NaN,20230203,...,1539.0,3.0,1,E,19,3,7.02,-12.06,3.42,14.367338
4,64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220905,...,2141.0,4.0,1,A,8,1,-9.18,-9.09,1.44,12.999004


In [45]:
df.to_csv("/content/drive/MyDrive/유나다 코드 정리/빅콘테스트/2. EDA/df_final.csv'")